<a href="https://colab.research.google.com/github/AkashKoley012/Deep-Learning-Projects/blob/main/Netflix%20Stock%20Price%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting Data

In [38]:
!kaggle datasets download -d jainilcoder/netflix-stock-price-prediction

Dataset URL: https://www.kaggle.com/datasets/jainilcoder/netflix-stock-price-prediction
License(s): CC0-1.0
netflix-stock-price-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [39]:
import zipfile
zip_ref = zipfile.ZipFile('/content/netflix-stock-price-prediction.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Imports modules

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Feature Engineering

In [41]:
df = pd.read_csv('/content/NFLX.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [42]:
df.shape

(1009, 7)

# Scaled Data

In [43]:
scaler = MinMaxScaler(feature_range=(-1,1))
data = scaler.fit_transform(df[['Close']].values)
data

array([[-0.91096748],
       [-0.86090301],
       [-0.86597063],
       ...,
       [-0.14549701],
       [-0.24981978],
       [-0.22985514]])

In [44]:
df['Close'].values

array([254.259995, 265.720001, 264.559998, ..., 429.480011, 405.600006,
       410.170013])

# Train Test

In [45]:
def prepare_data(data, seq_len):
  X = []
  y = []

  for i in range(len(data)-seq_len):
    X.append(data[i:(i+seq_len)])
    y.append(data[i+seq_len])

  return np.array(X), np.array(y)

In [46]:

X, y = prepare_data(data, seq_len=50)

In [47]:
X.shape, y.shape

((959, 50, 1), (959, 1))

In [48]:
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

In [49]:
X_train.shape, y_train.shape

((767, 50, 1), (767, 1))

# Dataset & DataLoader

In [68]:
class StockDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.tensor(X, dtype=torch.long)
    self.y = torch.tensor(y, dtype=torch.long)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [69]:
train_dataset = StockDataset(X_train, y_train)
test_dataset = StockDataset(X_test, y_test)

In [70]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [71]:
for X, y in train_dataloader:
  print(X.shape, y.shape)
  break

torch.Size([64, 50, 1]) torch.Size([64, 1])


# Neural Network

In [ ]:
class LSTM(nn.Module):
  def __init__(self):
    super().__init__()
    self.embedding = nn.Embedding(1, 128)
    self.lstm = nn.LSTM(128, 128, batch_first=True)
    self.fc = nn.Linear(128, 1)
  def forward(self, X):
    x = self.embedding(X)
    x, _ = self.lstm(x)
    x = self.fc(x)
    return x

# Model

In [58]:
lr = 0.01
epochs = 100

In [72]:
x=[1,2,3,4,5,6]
x[:,-1,:]

TypeError: list indices must be integers or slices, not tuple